In [1]:
# conda create --clone pyvizenv --name smallcapcryptosenv -y

In [2]:
# conda activate smallcapcryptosenv

In [3]:
# !conda install -c anaconda seaborn -y

In [4]:
# !pip install pycoingecko

In [5]:
# Initial Imports
import pandas as pd
import os
import requests
import numpy as np
# from dotenv import load_dotenv

from MCForecastTools import MCSimulation
import datetime as dt
import seaborn
import matplotlib.pyplot as plt
import panel as pn
from panel.interact import interact
import plotly.express as px

pn.extension("plotly")
%matplotlib inline

In [6]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [7]:
def time_range(delta=365):
	date_2 = pd.Timestamp("today", tz="America/Chicago") # current date as timestamp
	date_1 = date_2 - pd.to_timedelta(delta, unit="Days")
	
	dates = {
		"date_1": date_1,
		"date_2": date_2}
	return dates
	
dates = time_range()

In [8]:
def unix_date(dates):
	return [dates['date_1'].timestamp(), dates['date_2'].timestamp()]

In [9]:
dates = time_range()
dates

{'date_1': Timestamp('2021-04-02 13:54:55.865392-0500', tz='America/Chicago'),
 'date_2': Timestamp('2022-04-02 13:54:55.865392-0500', tz='America/Chicago')}

In [10]:
from_to = unix_date(dates)
from_to

[1617389695.865392, 1648925695.865392]

In [ ]:
# unix_date(dates)

In [ ]:
# dates = time_range()
# d1_unix = pd.to_datetime(dates["date_1"])#.timestamp()#.map(pd.Timestamp.timestamp)
# print(d1_unix.timestamp())
# dates[date_1.timestamp()]

In [11]:
maple = requests.get('https://api.coingecko.com/api/v3/coins/maple/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834').json()

In [12]:
# maple params list
p_list = {
	'ticker':'maple',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}

In [13]:
def make_url(p_list):
	my_url = f"https://api.coingecko.com/api/v3/coins/{p_list['ticker']}/{p_list['request']}?vs_currency={p_list['currency']}&from={p_list['date_1']}&to={p_list['date_2']}"
	return my_url

In [14]:
maple_url = make_url(p_list)
maple_url

'https://api.coingecko.com/api/v3/coins/maple/market_chart/range?vs_currency=usd&from=1617382350.549834&to=1648918350.549834'

In [15]:
maple_json = requests.get(maple_url).json()

In [16]:
maple_df = pd.DataFrame.from_dict(maple_json, orient='columns', dtype=None, columns=None)
maple_df

,prices,market_caps,total_volumes
0,"[1619740800000, 23.327418267811513]","[1619740800000, 0.0]","[1619740800000, 16576656.85415079]"
1,"[1619827200000, 23.327418267811513]","[1619827200000, 0.0]","[1619827200000, 16576656.85415079]"
2,"[1619913600000, 18.915452769785436]","[1619913600000, 0.0]","[1619913600000, 31537632.939870752]"
3,"[1620000000000, 21.309982603593916]","[1620000000000, 0.0]","[1620000000000, 1009195.8446439038]"
4,"[1620086400000, 17.993651917843852]","[1620086400000, 0.0]","[1620086400000, 699889.2299604575]"
...,...,...,...
333,"[1648512000000, 44.45836995899958]","[1648512000000, 205328245.69898155]","[1648512000000, 11288587.509875141]"
334,"[1648598400000, 46.561385905260124]","[1648598400000, 220744734.5592906]","[1648598400000, 7550300.014392077]"
335,"[1648684800000, 52.24895192787693]","[1648684800000, 249319778.6861569]","[1648684800000, 10566379.186666392]"
336,"[1648771200000, 50.970282320420615]","[1648771200000, 242426811.27108622]","[1648771200000, 12544229.03797571]"


In [17]:
# bitcoin params list
p_list = {
	'ticker':'bitcoin',
	'request':'market_chart/range',
	'currency':'usd',
	'date_1':1617382350.549834,
	'date_2':1648918350.549834,
}

In [18]:
bitcoin_url = make_url(p_list)
bitcoin_json = requests.get(bitcoin_url).json()
bitcoin_df = pd.DataFrame.from_dict(bitcoin_json, orient='columns', dtype=None, columns=None)

In [19]:
bitcoin_df

,prices,market_caps,total_volumes
0,"[1617408000000, 59060.02678333801]","[1617408000000, 1103133931136.5508]","[1617408000000, 52224918752.08043]"
1,"[1617494400000, 57060.40665471088]","[1617494400000, 1067101827372.9529]","[1617494400000, 52067853987.600494]"
2,"[1617580800000, 58228.802623466196]","[1617580800000, 1087344280109.6896]","[1617580800000, 46156767978.177345]"
3,"[1617667200000, 58706.81559311396]","[1617667200000, 1096202611389.9541]","[1617667200000, 54670173277.08731]"
4,"[1617753600000, 58062.17872433649]","[1617753600000, 1084341670320.8142]","[1617753600000, 61497704158.13829]"
...,...,...,...
360,"[1648512000000, 46994.879118357465]","[1648512000000, 892747620147.7012]","[1648512000000, 30051327357.066643]"
361,"[1648598400000, 47459.261237539096]","[1648598400000, 901430001888.9999]","[1648598400000, 26128040680.825665]"
362,"[1648684800000, 47063.36584996355]","[1648684800000, 895356010540.8462]","[1648684800000, 24497248306.346912]"
363,"[1648771200000, 45528.40715313835]","[1648771200000, 865010142444.9791]","[1648771200000, 27736469645.537262]"
